In [3]:
import numpy as np 

#Definindo uma semente geradora de número aleatória
np.random.seed(123456)
path_out = 'saidaSim.txt' #Caminho do arquivo a ser gerado

#Parametrização das distribuições gauss. mult. usadas
mu1 = [1.0, 1.0]; Sigma1 = [[0.75, 0.5], [0.5,2.0]]
rotulo1 = 1; qnt1 = 50
mu2 = [-2.0, 2.0]; Sigma2 = [[0.5, -0.5], [-0.5, 1]]
rotulo2 = 2; qnt2 = 40

#Construção de arrays de parametros/rotulos/quantidades
conjMu = np.array([mu1,mu2])
conjSigma = np.array([Sigma1, Sigma2])
rotulos = np.array([rotulo1,rotulo2])
quantidades = np.array([qnt1, qnt2])

#Geração de arquivo de saída
f = open(path_out, 'w')
for r in range(rotulos.size):
    #Confira documentação da função multivariada_normal
    #Transposição '.T' usada para fins de computalização
    dados = np.random.multivariate_normal(conjMu[r,:], conjSigma[r,:,:], quantidades[r]).T

    for i in range(0, quantidades[r]):
        st = str(rotulos[r])
        for j in range(0, dados.shape[0]):
            st += ','+ str(dados[j,i])
        st += '\n' #Força continuação na próxima linha
        f.write(st)

f.close() #Encerra o processo de escrita dos dados